In [ ]:


import numpy as np
from sklearn.neighbors import NearestNeighbors 
from sklearn.cluster import SpectralClustering
from sklearn.cluster import OPTICS
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from scipy.spatial import ConvexHull
#import plotly.express as px
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d
from concurrent.futures import ProcessPoolExecutor
from scipy.stats import linregress
from sklearn.metrics import r2_score
import time
import os
from concurrent import futures
# os.environ["OMP_NUM_THREADS"] = "20" 
from yellowbrick.cluster import KElbowVisualizer
import matplotlib.pyplot as plt
# %matplotlib qt
# %matplotlib inline
# %matplotlib notebook

In [ ]:
data=np.genfromtxt(r'Enter File Path',skip_header=1)

In [ ]:
import numpy as np

# Assuming your data is stored in a numpy array called 'data'

# Extracting the normal vectors
normals = data[:, 6:9]

# Removing the columns from 7th to 9th
data = np.delete(data, np.s_[6:9], axis=1)

# Computing the angles subtended by the normal vectors in the xy plane and xz plane
xy_plane_normal = np.array([0, 0, 1])  # Normal vector perpendicular to the xy plane
xz_plane_normal = np.array([0, 1, 0])  # Normal vector perpendicular to the xz plane
yz_plane_normal = np.array([1, 0, 0])  # Normal vector perpendicular to the xz plane

# def compute_angle(vector, normal):
#     projection = vector - np.dot(vector, normal) * normal
#     return np.degrees(np.arctan2(np.linalg.norm(np.cross(normal, projection)), np.dot(normal, projection)))

def compute_angle(vector1, vector2):
    unit_vector1 = vector1 / np.linalg.norm(vector1)
    unit_vector2 = vector2 / np.linalg.norm(vector2)
    dot_product = np.dot(unit_vector1, unit_vector2)
    angle_radians = np.arccos(dot_product)
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees

# Sequential angle calculations
angle_xy = []
angle_xz = []
angle_yz = []

for i in range(normals.shape[0]):
    angle_xy.append(compute_angle(normals[i], xy_plane_normal))
    angle_xz.append(compute_angle(normals[i], xz_plane_normal))
    angle_yz.append(compute_angle(normals[i], yz_plane_normal))

angle_xy = np.array(angle_xy)
angle_xz = np.array(angle_xz)
angle_yz = np.array(angle_yz)

# Adding the new columns for angle_xy and angle_xz in place of the removed columns
data = np.insert(data, 6, angle_xy, axis=1)
data = np.insert(data, 7, angle_xz, axis=1)
data = np.insert(data, 8, angle_yz, axis=1)


In [ ]:
# for i in range(len(data[:,6:9])):
#     print(data[i,6:9])
# unique_values=np.unique(data[:,2],axis=0)    
# unique_values=np.sort(unique_values)
# for i in unique_values:
#     fig = plt.figure()
#     ax = fig.add_subplot(projection='3d')
#     point_cloud=data[i==data[:,2]]
#     ax.scatter(point_cloud[:,6],point_cloud[:,7],point_cloud[:,8])
#     plt.show()

In [ ]:
def normal_cluster(X,clusterNumbers):
    clustering = OPTICS(min_samples=2).fit(X)  #performing spectral clustering on normal data
    # clustering=KElbowVisualizer(KMeans(),k=(1,3)).fit(X)
    return clustering.labels_

def xyz_cluster(X,clusterNumbers):
    unique_normal=np.unique(X[:,9])  #getting different labels
    # X=np.append(X,np.zeros((X.shape[0],1)),axis=1)  #making column for xyz labels   
    xyz_labels=[]
    for i in unique_normal:
        if X[i== X[:,9]][:,:2].shape[0]>=clusterNumbers: #take care of this line in future(agar point kam ho toh uss wale normal pe outliers ho skate hain and they might be the corners or important point)
            
            clustering = OPTICS(min_samples=2).fit(X[i== X[:,9]][:,:2]) #spectral clustering performed on xyz of point cloud data
        # X[i==X[:,9]][:,10]=clustering.labels_  #changing values from zero to actual xyz labels 
            xyz_labels=xyz_labels + list(clustering.labels_)
        else:
            xyz_labels=xyz_labels + list(0 for x in range(X[i== X[:,9]][:,:2].shape[0]))
    xyz_labels=np.asanyarray(xyz_labels)
    X=np.append(X,xyz_labels.reshape((-1,1)),axis=1)
    return X

In [ ]:
def linear_regression(X,iterator,axis):
    global data
    if axis==0:
        axis_0=1
        axis_1=2
    elif axis==1:
        axis_0=0
        axis_1=2
    elif axis==2:
        axis_0=0
        axis_1=1
    unique_normals=np.unique(X[:,9])
    # iteration=0
    line=np.empty((0,10))
    for i in unique_normals:
        # print(i)
        # fig=px.scatter_3d(X[i==X[:,9]],x=0,y=1,z=2,color=3)
        # fig.show()

        points=X[i==X[:,9]]
        unique_xyz=np.unique(points[:,10])
        for j in unique_xyz:

            points_sorted=points[j==points[:,10]][points[j==points[:,10]][:,axis_0].argsort()]

            try:
                result=linregress(points_sorted[:,axis_0], points_sorted[:,axis_1])
                result=result.intercept + result.slope*(points_sorted[:,axis_0].reshape((-1,1)))
                r_square=r2_score(points_sorted[:,axis_1].reshape((-1,1)),result)

            except:
                points_sorted=points[j==points[:,10]][points[j==points[:,10]][:,axis_1].argsort()]
                r_square=0
                
            if r_square<0.5 and r_square!=0:
                                                                                   #that means given points form a plane and hence we will find it convex hull
                try:
                    hull=ConvexHull(points_sorted[:,(axis_0,axis_1)])
                except:
                    continue
                line_coordinates=points_sorted[hull.vertices][:,:9]
                line_coordinates=np.concatenate((line_coordinates[0].reshape((1,9)),np.repeat(line_coordinates[1:-1,:],2,0),line_coordinates[-1].reshape((1,9))),axis=0)
                line_coordinates=np.append(line_coordinates,np.repeat(np.arange(iterator,(line_coordinates.shape[0]/2)+iterator),2).reshape((-1,1)),axis=1)
                line=np.append(line,line_coordinates,axis=0)
                

            else: 
                low_x,high_x=points_sorted[0,0],points_sorted[-1,0]                #getting lowest and highest point on x axis to make a line
                low_z,high_z=points_sorted[0,2],points_sorted[-1,2]                #getting the z values of lowest and highest  points 
                line_coordinates=np.array([ [low_x,high_x], [points_sorted[0,1],points_sorted[-1,1]],[low_z,high_z],[points_sorted[0,3],points_sorted[-1,3]],[points_sorted[0,4],points_sorted[-1,4]],[points_sorted[0,5],points_sorted[-1,5]],[points_sorted[0,6],points_sorted[-1,6]],[points_sorted[0,7],points_sorted[-1,7]],[points_sorted[0,8],points_sorted[-1,8]]],dtype=object).T  #drawing a line
                line_coordinates=np.append(line_coordinates,[[iterator],[iterator]],axis=1)
                data[(np.where((data[:,:3] == line_coordinates[0, :3]).all(axis=1))[0]),3:6]=np.array([1,1,1]).reshape((1,3))

                line=np.append(line,line_coordinates,axis=0)
            
            iterator+=1

    return  line

In [ ]:
iterator_x=0
iterator_y=pow(10,5)
iterator_z=pow(10,7)

In [ ]:

start=time.perf_counter()

def process_slices(slice_value,iterator,axis):

    global data
    lines=np.empty((1,10))

    X=data[slice_value==data[:,axis]]

    if X.shape[0]>2:
        normal_cluster_number=4
        normal_clustered_labels= normal_cluster(X[:,6:],normal_cluster_number)        #calling normal_cluster function
        X= np.append(X,normal_clustered_labels.reshape((-1,1)),axis= 1)               #appending label column to pointcloud data

        xyz_cluster_number=1
        X= xyz_cluster(X,xyz_cluster_number)                                          #calling xyz cluster function
        print('xyz clutering completed')

        line_coordinates=linear_regression(X,iterator,axis)                                #calling linear regression function
        lines=np.append(lines,line_coordinates,axis=0)

        lines=lines[1:]
        return lines
    else:
        return np.array([[X[0,0],X[0,1],X[0,2],iterator]])

def get_slices(axis):
    global data
    if axis==0:
        global iterator_x
        iterator_start=iterator_x
    elif axis==1:
        global iterator_y
        iterator_start=iterator_y
    elif axis==2:
        global iterator_z
        iterator_start=iterator_z

    data[:,axis]=data[:,axis].round(decimals=3)
    unique_values=np.unique(data[:,axis],axis=0)    
    unique_values=np.sort(unique_values)
    iterator=np.arange(iterator_start,20*unique_values.shape[0]+iterator_start,20)

    axis=[axis for i in range(unique_values.shape[0])]
    os.environ["OMP_NUM_THREADS"] = "1"                                                #using only one thread for numpy arrays for time manangement
    with ProcessPoolExecutor(max_workers=38) as executor:
        results=executor.map(process_slices,unique_values,iterator,axis)
    wireframe=np.empty((1,10))
    for result in results:
        temp=result
        wireframe=np.concatenate((wireframe,np.array(temp)),axis=0)
    wireframe=wireframe[1:]
    return wireframe 

axis=[0,2]
with ProcessPoolExecutor(max_workers=1) as executor:
    results=executor.map(get_slices,axis)


print(time.perf_counter()-start)
# lines=lines[1:]


In [ ]:
wireframe=np.empty((1,10))
for result in results:
    temp=result
    wireframe=np.concatenate((wireframe,np.array(temp)),axis=0)
wireframe=wireframe[1:]
# wireframe.shape

In [ ]:
fig = plt.figure()
ax = plt.axes(projection="3d")
ax.scatter3D(wireframe[:,0],wireframe[:,1],wireframe[:,2])
ax.view_init(0,-15)
plt.show()